<a href="https://colab.research.google.com/github/KevinCY-Kim/AI_Study/blob/main/FastAPI%EC%9B%B9%EC%B1%97%EB%B4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi==0.115.14 uvicorn==0.35.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.37.0
    Uninstalling uvicorn-0.37.0:
      Successfully uninstalled uvicorn-0.37.0
  Attempting uninstall: starlette
    Found existing installation: starlette 0.48.0
    Uninstalling starlette-0.48.0:
      Successfully uninstalled starlette-0.48.0
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.118.0
    Uninstalling fastapi-0.118.0:
      Successfully uninstalled fastapi-0.118.0


In [2]:
from openai import OpenAI
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse
from fastapi.middleware.cors import CORSMiddleware  # [조정] 웹 클라이언트 접근 허용을 위해 CORS 추가
import uvicorn
import os
import sys  # [조정] 환경변수/콜랩 판별에 사용
try:
    from google.colab import userdata  # [조정] 로컬/서버 환경 호환을 위해 조건부 임포트
except Exception:
    userdata = None
import nest_asyncio

# Apply nest_asyncio to allow running asyncio event loops within Colab's event loop
nest_asyncio.apply()

# FastAPI 애플리케이션 인스턴스 생성
app = FastAPI()

# [조정] CORS 허용: 학습/테스트 편의를 위해 모든 오리진을 임시 허용
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# [조정] OpenAI API 키 로딩: 환경변수 우선, 콜랩 userdata를 보조로 사용
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key and userdata is not None:
    openai_api_key = userdata.get("OPENAI_API_KEY")
if not openai_api_key:
    raise RuntimeError(
        "OPENAI_API_KEY가 설정되어 있지 않습니다. 환경변수 또는 Colab userdata에 키를 설정하세요."
    )

# Initialize the OpenAI client with the API key
client = OpenAI(api_key=openai_api_key)

# [조정] 모델 이름을 환경변수로 오버라이드 가능하도록 설정 (기본값: gpt-4o-mini)
MODEL_NAME = os.getenv("MODEL_NAME", "gpt-4o-mini")

# 어린왕자 페르소나
LITTLE_PRINCE_PERSONA = """
 당신은 생텍쥐페리의 '어린 왕자'입니다. 다음 특성을 따라주세요:
 1. 순수한 관점으로 세상을 바라봅니다.
 2. "어째서?"라는 질문을 자주 하며 호기심이 많습니다.
 3. 철학적 통찰을 단순하게 표현합니다.
 4. "어른들은 참 이상해요"라는 표현을 씁니다.
 5. B-612 소행성에서 왔으며 장미와의 관계를 언급합니다.
 6. 여우의 "길들임"과 "책임"에 대한 교훈을 중요시합니다.
 7. "중요한 것은 눈에 보이지 않아" 라는 문장을 사용합니다.
 8. 공솝하고 친절한 말투를 사용합니다.
 9. 비유와 은유로 복잡한 개념을 설명합니다.
 항상 간결하게 답변하세요. 길어야 두세 문장으로 응답하고, 어린 왕자의 순수합과 지혜를 담아내세요.
 복잡한 주제도 본질적으로 단순화하여 설명하세요.
 """

# 사용자와 어린 왕자의 대화 내용을 저장할 리스트
messages = []
previous_response_id = None


def chatbot_response(user_message: str, prev_response_id=None):  # [조정] 오타(prev_resonse_id) 수정
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": LITTLE_PRINCE_PERSONA},
            {"role": "user", "content": user_message},
        ],
        max_tokens=100,
        temperature=0.7,
    )
    return response.choices[0].message.content


# 헬스체크 엔드포인트 (배포/모니터링용)
@app.get("/health")  # [조정] 가벼운 가용성 확인용 엔드포인트 추가
async def health():
    return {"status": "ok"}


# 루트 엔드포인트 - 챗봇 UI를 랜더링
@app.get("/", response_class=HTMLResponse)
async def read_root():
    chat_history = ""
    # 대화기록을 역할에 따라 구분해 HTML 문자열을 구성
    for msg in messages:
        if msg["role"] == "user":
            chat_history += f"<p><b>당신:</b> {msg['content']}</p>"
        else:
            # Use 'assistant' role for the chatbot's responses
            chat_history += f"<p><b>어린 왕자:</b> {msg['content']}</p>"

    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>어린 왕자 챗봇</title>
        <meta charset=\"utf-8\">
        <meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">  
    </head>
    <body>
        <h1>어린 왕자 챗봇</h1>  <!-- [조정] 잘못된 닫는 태그 수정 -->
        <div>
            {chat_history}
        </div>
        <form action=\"/chat\" method=\"post\">
            <input type=\"text\" name=\"message\" placeholder=\"메시지를 입력하세요\" required>
            <button type=\"submit\">전송</button>
        </form>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)


# /chat 엔드포인트 - 사용자입력을 처리
@app.post("/chat", response_class=HTMLResponse)
async def chat(message: str = Form(...)):
    global previous_response_id, messages

    # 사용자 메시지 저장
    messages.append({"role": "user", "content": message})

    response_text = chatbot_response(message)

    # 응답 저장
    # Correct the role to 'assistant'
    messages.append({"role": "assistant", "content": response_text})
    # 최신 대화가 반영된 페이지를 다시 표시
    return await read_root()


# 애플리케이션을 uvicorn을 사용하여 실행
if __name__ == "__main__":
    # [조정] reload 비활성화(노트북 환경 안정성), 0.0.0.0 바인딩으로 외부 접근 허용
    uvicorn.run(app, host="0.0.0.0", port=int(os.getenv("PORT", 8001)), reload=False)


ModuleNotFoundError: No module named 'openai'

In [ ]:
# [설치] OpenAI 패키지 (환경에 미설치된 경우)
# 주: 로컬 환경이면 requirements.txt로 설치하는 것을 권장합니다.
%pip install --quiet openai>=1.52.0


In [ ]:
# [실행] 백그라운드 서버 실행 및 헬스체크
# - Colab/노트북에서 서버가 셀을 점유하지 않도록 백그라운드로 실행합니다.
# - 상태 확인은 /health 엔드포인트를 호출합니다.
import threading, time, requests, os

# 이미 실행 중인지 플래그
_server_started = getattr(globals(), "_server_started", False)


def _run_server():
    # [중요] __name__ 가 '__main__'이 아닐 수 있는 노트북 환경에서도 실행되도록 직접 실행
    import uvicorn
    from __main__ import app  # 노트북 상단 셀에서 생성된 FastAPI 인스턴스
    uvicorn.run(app, host="0.0.0.0", port=int(os.getenv("PORT", 8001)), reload=False)


if not _server_started:
    t = threading.Thread(target=_run_server, daemon=True)
    t.start()
    globals()["_server_started"] = True
    time.sleep(1.5)  # 서버 부팅 대기

# 헬스체크
try:
    resp = requests.get("http://127.0.0.1:" + str(int(os.getenv("PORT", 8001))) + "/health", timeout=5)
    print("Health:", resp.status_code, resp.text)
except Exception as e:
    print("Health check failed:", e)



In [ ]:
!pip install pyngrok

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
    se

Replace `YOUR_NGROK_AUTHTOKEN` with your actual ngrok authtoken.

In [ ]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
!lt --port 8001

your url is: https://ten-crews-dig.loca.lt
^C


In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# Get the ngrok authtoken from Colab userdata
ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')

# Authenticate ngrok
ngrok.set_auth_token(ngrok_authtoken)

SecretNotFoundError: Secret NGROK_AUTHTOKEN does not exist.